In [1]:
#The file needed to run this code is Data_Group7_data.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('/kaggle/input/data304-project-group7/Data_Group7_data.csv', index_col=0)

# the Arrivals in df, some of the value is in hh:mm:ss format, so we need to convert it to hh:mm, delete the second
# df['Arrivals'] = df['Arrivals'].str[:-3]

for i in range(len(df['Arrivals'])):
    # if the length of the value is larger than 5 but less than 8, then we need to delete the second
    if len(df['Arrivals'][i]) > 5 and len(df['Arrivals'][i]) <= 8:
        df['Arrivals'][i] = df['Arrivals'][i][:-3]
    # else if the length of the value is larger than 8, then we need to delete the second and the PM
    if len(df['Arrivals'][i]) > 8:
        df['Arrivals'][i] = df['Arrivals'][i][:-6]
        

for i in range(len(df['Departures'])):
    # if the length of the value is larger than 5 but less than 8, then we need to delete the second
    if len(df['Departures'][i]) > 5 and len(df['Departures'][i]) <= 8:
        df['Departures'][i] = df['Departures'][i][:-3]
    # else if the length of the value is larger than 8, then we need to delete the second and the PM
    if len(df['Departures'][i]) > 8:
        df['Departures'][i] = df['Departures'][i][:-6]
        
df[["Arrivals","Departures"]] = df[["Arrivals","Departures"]].astype('datetime64[ns]')
df["service_time"] =  df["Departures"] - df["Arrivals"]
time_parts = df["service_time"].apply(lambda x: pd.Series(str(x).split(":")))  # split the time string into hours, minutes, and seconds

df["interarrivals"] = df["Arrivals"] - df["Arrivals"].shift(1)
time_parts_2 = df["interarrivals"].apply(lambda x: pd.Series(str(x).split(":")))  # split the time string into hours, minutes, and seconds

In [2]:
df['Arrivals'] = pd.to_datetime(df['Arrivals'], format='%H:%M', errors='coerce')
df['Departures'] = pd.to_datetime(df['Departures'], format='%H:%M', errors='coerce')

df.dropna(subset=['Arrivals', 'Departures'], inplace=True)

#Here we obtain a list of our times in seconds, for use in our models later on
times = pd.to_datetime(df['Arrivals']).dt.time
arrival_times = []
for time in times:
  h = time.hour
  m = time.minute
  arrival_times.append(int(h)*3600 +int(m*60))

#Here we obtain the interarrival times and rate
interarrival_times = df['Arrivals'].diff().dt.total_seconds().dropna()
interarrival_rate = np.mean(interarrival_times)/60
#Here we get the service times as a time difference
service_times = (df['Departures']-df['Arrivals']).astype('timedelta64[m]')



In [3]:
#This code is for our mu and lambda
print("Mean Service Time/mu: ",np.mean(service_times))
print("Mean interarrival rate/lambda: ",interarrival_rate)
lamb = interarrival_rate
mu = np.mean(service_times)


Mean Service Time/mu:  -51.48943661971831
Mean interarrival rate/lambda:  1.0106007067137808


In [4]:
""""Model 1: M/M/1 queueing system with multiple replications"""
import sys
!{sys.executable} -m pip install SimPyClassic
from SimPy.Simulation import *
import random
import numpy
import math
"""Extras"""
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return lower, upper
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(mu))
            t = random.expovariate(lamb)
            yield hold, self, t
class Arrival(Process):
    """an arrival"""
    def run(self, mu):
        Arrival.n += 1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
  
        yield request, self, G.server
        
        t = random.expovariate(1/mu) #1 person per mu time
        
        yield hold, self, t
        
        yield release, self, G.server
        
        Arrival.n-=1
        
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)   
class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon='Monitor'
    busymon='Monitor'
def model(c, N, lamb, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon= Monitor()
    Arrival.n = 0
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, mu))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon.mean()
    L= G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    
    return W,L,B
## Experiment ----------
allW = []
allL = []
allB = []
for k in range(50):
    seed = 123*k
    result = model(c=1, N=100, lamb=1.0106007067137808, mu=9.355633802816902,
                  maxtime=180, rvseed=seed) #The mu and lambda defined here are calculated earlier in the code too
    allW.append(result[0])
    allL.append(result[1])
    allB.append(result[2])
print("Estimate of W:", numpy.mean(allW))
print("Conf int of W:", conf(allW))
print("Estimate of L:", numpy.mean(allL))
print("Conf int of L:", conf(allL))
print("Estimate of B:", numpy.mean(allB))
print("Conf int of B:", conf(allB))    

""" Our queue system recorded data only at peak times for the Lab. Because of this, our queue system in an M/M/1 queue is not stable. Thus, the queue length increases exponentially"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.7 MB/s eta 0:00:00
Estimate of W: 78.29437525860833
Conf int of W: (74.71993174525343, 81.86881877196322)
Estimate of L: 62.33050166862729
Conf int of L: (60.91526702986899, 63.74573630738559)
Estimate of B: 0.998708084592924
Conf int of B: (0.9977432220047133, 0.9996729471811348)


' Our queue system recorded data only at peak times for the Lab. Because of this, our queue system in an M/M/1 queue is not stable. Thus, the queue length increases exponentially'

In [5]:
"""Model 2: Best Fit Model """
from SimPy.Simulation import *
import random
import numpy as np
import math
import scipy
import scipy.stats as st

def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper
#Using Pearson fit as it works the best for our arrival times
P3skew,P3loc,P3scale = st.pearson3.fit(arrival_times)
print(P3skew,P3loc,P3scale)

#This fixes the service times and provides a list of service times for easy use
fixed_service_times = []
for service in service_times:
  if(service > 0):
    fixed_service_times.append(service)

#Beta fit is a useful model for our service times
beta_a, beta_b,beta_loc,beta_scale = st.beta.fit(fixed_service_times)
print(beta_a,beta_b,beta_loc,beta_scale)

class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = abs(st.pearson3.rvs(skew=P3skew,loc=P3loc,scale=P3scale))
            yield hold, self, t

class Arrival(Process):
    """an arrival"""
    n=0 
    def run(self):
        Arrival.n +=1
        
        arrivetime = now()
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        yield request, self, G.server
        yield hold, self, st.beta.rvs(a=beta_a,b=beta_b,loc=beta_loc,scale=beta_scale)
        yield release, self, G.server
        
        Arrival.n -=1
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        delay = now()-arrivetime
        G.delaymon.observe(delay)

class G:
    server = 'dummy'
    delaymon = 'Monitor' #W value
    numbermon = "Monitor" #L value
    busymon = 'Monitor' #B value

def model(N, maxtime, rvseed,K):
    #Setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(K)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()

    Arrival.n = 0
    
    #Simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    #Gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    
    return W, L, B
for Kap in [3]:
    allW = []
    allL = []
    allB = []
    allLambdaEffective = []

    for k in range(50):
        seed = 123*k
        result = model(N=10000, maxtime=200, rvseed=seed, K=Kap)
        allW.append(result[0])
        allL.append(result[1])
        allB.append(result[2])
        allLambdaEffective.append(result[1]/result[0])

    print("When K=",Kap)
    print("    Estimate of W:", np.mean(allW))
    print("    Conf int of W:", conf(allW))
    print("    Estimate of L:", np.mean(allL))
    print("    Conf int of L:", conf(allL))
    print("    Estimate of B:", np.mean(allB))
    print("    Conf int of B:", conf(allB))
    print("    Estimate of LambdaEffective:", np.mean(allLambdaEffective))
    print("    Conf int of LambdaEffective:", conf(allLambdaEffective))
   

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.5247359326279972 44164.01395039915 3827.950204878982
398.92925389176696 6.022880183615848 -371.42581458658447 386.16516674675023
When K= 3
    Estimate of W: 8.947157769187259
    Conf int of W: (8.38874317633976, 9.505572362034757)
    Estimate of L: 0.044735788845936285
    Conf int of L: (0.0419437158816988, 0.04752786181017377)
    Estimate of B: 0.044735788845936285
    Conf int of B: (0.0419437158816988, 0.04752786181017377)
    Estimate of LambdaEffective: 0.005
    Conf int of LambdaEffective: (0.005, 0.005)


/opt/conda/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


In [6]:
"""Model 3: Empirical"""
import numpy as np
import random
import math
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import sys
!{sys.executable} -m pip install SimPyClassic
from SimPy.Simulation import *

def draw_empirical(data, r):
  """one draw (for given r ~ U(0,1)) from the empirical cdf based on data"""
  d = {x:data.count(x) for x in data}
  obs_values, freq = zip( *sorted( zip(d.keys(), d.values())))
  obs_values = list(obs_values)
  freq = list(freq)
  empf = [x*1.0/len(data) for x in freq]
  ecum = np.cumsum(empf).tolist()
  ecum.insert(0, 0)
  obs_values.insert(0,0)

  for x in ecum:
    if r <= x:
      rpt = x
      break

  r_end = ecum.index(rpt)
  y = obs_values[r_end] - 1.0*(ecum[r_end]-r)*(obs_values[r_end]-obs_values[r_end-1])/(ecum[r_end]-ecum[r_end-1])
  return y

fixed_service_times = []
for service in service_times:
  if(service > 0):
    fixed_service_times.append(service)

times = pd.to_datetime(df['Arrivals']).dt.time
arrival_times = []
for time in times:
  h = time.hour
  m = time.minute
  arrival_times.append(int(h)*3600 +int(m*60))

# draw_empirical(arrival_times, random.random())
# draw_empirical(fixed_service_times, random.random())

def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper
class Source(Process):
    """generate random arrivals"""
    def run(self, N, arrival, service):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(service))
            t = draw_empirical(arrival,random.random())
            yield hold, self, t
class Arrival(Process):
    """an arrival"""
    def run(self, service):
        Arrival.n += 1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0) 
        yield request, self, G.server
        
        t = draw_empirical(service,random.random())  
        yield hold, self, t       
        yield release, self, G.server
        
        Arrival.n-=1
        
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)   
class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = "Monitor" 
    busymon = 'Monitor' 
def model(c, N, arrival, service, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()
    Arrival.n = 0

    # simulate
    s = Source('Source')
    activate(s, s.run(N, arrival, service))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()

    return W, L, B
## Experiment ----------------

allW = []
allL = []
allB = []
allLambdaEffective = []

for k in range(50):
    seed = 123*k
    result = model(c = 1, N=10000, arrival = arrival_times, service = fixed_service_times, maxtime=27000, rvseed=seed)
    allW.append(result[0])
    allL.append(result[1])
    allB.append(result[2])
    allLambdaEffective.append(result[1]/result[0])

print("Estimate of W:", np.mean(allW))
print("Conf int of W:", conf(allW))
print("Estimate of L:", np.mean(allL))
print("Conf int of L:", conf(allL))
print("Estimate of B:", np.mean(allB))
print("Conf int of B:", conf(allB))

Estimate of W: 7.935988220503076
Conf int of W: (7.100692816317219, 8.771283624688934)
Estimate of L: 0.00029392548964826204
Conf int of L: (0.0002629886228265636, 0.00032486235646996046)
Estimate of B: 0.00029392548964826204
Conf int of B: (0.0002629886228265636, 0.00032486235646996046)
